In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
%cd C:\Users\asche\deep_learning\exam-project\kunskapskontroll_2

C:\Users\asche\deep_learning\exam-project\kunskapskontroll_2


In [4]:
from flask import Flask, render_template, request, Response, jsonify
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import base64

In [5]:
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
emotion_model = load_model('final_model.h5', compile=False)

### Face detection app which opens a window with the local webcam.
Draws a rectangle on detected faces.

In [6]:
'''


import cv2
import sys
import logging as log
import datetime as dt
from time import sleep

cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
log.basicConfig(filename='webcam.log',level=log.INFO)

video_capture = cv2.VideoCapture(0)
anterior = 0

while True:
    if not video_capture.isOpened():
        print('Unable to load camera.')
        sleep(5)
        pass

    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    if anterior != len(faces):
        anterior = len(faces)
        log.info("faces: "+str(len(faces))+" at "+str(dt.datetime.now()))


    # Display the resulting frame
    cv2.imshow('Video', frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Display the resulting frame
    cv2.imshow('Video', frame)

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

'''

'\n\n\nimport cv2\nimport sys\nimport logging as log\nimport datetime as dt\nfrom time import sleep\n\ncascPath = "haarcascade_frontalface_default.xml"\nfaceCascade = cv2.CascadeClassifier(cascPath)\nlog.basicConfig(filename=\'webcam.log\',level=log.INFO)\n\nvideo_capture = cv2.VideoCapture(0)\nanterior = 0\n\nwhile True:\n    if not video_capture.isOpened():\n        print(\'Unable to load camera.\')\n        sleep(5)\n        pass\n\n    # Capture frame-by-frame\n    ret, frame = video_capture.read()\n\n    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n    faces = faceCascade.detectMultiScale(\n        gray,\n        scaleFactor=1.1,\n        minNeighbors=5,\n        minSize=(30, 30)\n    )\n\n    # Draw a rectangle around the faces\n    for (x, y, w, h) in faces:\n        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)\n\n    if anterior != len(faces):\n        anterior = len(faces)\n        log.info("faces: "+str(len(faces))+" at "+str(dt.datetime.now()))\n\n\n    # Di

### Face detection app which opens a browser. A website displays the local webcam.
Draws a rectangle on detected faces.

In [ ]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

def detect_faces():
    while True:
        ret, frame = video_capture.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        _, jpeg = cv2.imencode('.jpg', frame)
        frame_bytes = jpeg.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(detect_faces(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
